### Retrival Documents
* Step 1 - Load documents
* Step 2 - Split documets
* Step 3 - Encode and store in a vectorstore
* Step 4 - Retrival

#### Load Documents             
* Langchain have multiple document loaders buildin which loads a document into **langchain_core.documents.base.Document** type.          
* Langchain have several inbulit document loaders - 
    * CSV      
    * HTML         
    * JSON        
    * Markdown      
    * PDF      
    * Microsoft

In [20]:
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document
from typing import Iterator

class TextDocumentLoader(BaseLoader):
    def __init__(self, path:str):
        self.path = path     #file path
    def lazy_load(self) -> Iterator[Document]:
        with open(self.path, encoding='utf=8') as file:
            for i,line in enumerate(file):
                yield Document(page_content=line,metadata={'line_number':i})
    def load(self) -> list[Document]:
        return super().load()
    def full_load(self) -> Document:
        with open(self.path, encoding='utf-8') as file:
            text = file.read()
            return Document(page_content=text, metadata={'file_path':self.path})

In [ ]:
from pathlib import Path

file_path = Path("text.txt")
loader = TextDocumentLoader(file_path).lazy_load()
for doc in loader:
    print(doc)

page_content='Hello this is line 1.
' metadata={'line_number': 0}
page_content='This is the 2nd line of the document.
' metadata={'line_number': 1}
page_content='This is the 3rd line of the document.
' metadata={'line_number': 2}
page_content='This is the 4th and the final line of the document.' metadata={'line_number': 3}


In [17]:
TextDocumentLoader(file_path).load()

[Document(metadata={'line_number': 0}, page_content='Hello this is line 1.\n'),
 Document(metadata={'line_number': 1}, page_content='This is the 2nd line of the document.\n'),
 Document(metadata={'line_number': 2}, page_content='This is the 3rd line of the document.\n'),
 Document(metadata={'line_number': 3}, page_content='This is the 4th and the final line of the document.')]

In [22]:
document = TextDocumentLoader(file_path).full_load()

#### Text Splitter              
* After a document is loaded the document must be split into text chunks.             
* Different statergies can be used to split the text into different chunks.           
* Most common text splitters -      
    * Character chunker        
    * Sematic Chucking

In [2]:
#generator function
def count_to_n(n):
    count = 1
    while count<n+1:
        yield count
        count += 1

counter = count_to_n(10)
type(counter)

generator

In [3]:
for count in counter:
    print(f"count = {count}")
    print("Break")

count = 1
Break
count = 2
Break
count = 3
Break
count = 4
Break
count = 5
Break
count = 6
Break
count = 7
Break
count = 8
Break
count = 9
Break
count = 10
Break
